In [21]:
import os
import collections
import shutil
import json
import numpy as np

## Count the total number of files

Given the path to a folder which may contain subfolders, count the total number of files in this folder.

In [2]:
def get_number_of_files(path):
    files = os.listdir(path)
    num_of_files = 0
    for file in files:
        path_to_file = os.path.join(path, file)
        if os.path.isdir(path_to_file):
            num_of_files += get_number_of_files(path_to_file)
        else:
            num_of_files += 1
    return num_of_files

In [3]:
print(get_number_of_files("CN"))

596457


In [4]:
print(get_number_of_files("CN/A001"))

8129


## Get the name of all files

Given the path to a folder which may contain subfolders, get the name of all files in this folder.

In [5]:
def get_filename(path):
    filenames = []
    get_filename_help(path, filenames)
    return filenames

def get_filename_help(path, filenames):
    files = os.listdir(path)
    for file in files:
        path_to_file = os.path.join(path, file)
        if os.path.isdir(path_to_file):
            get_filename_help(path_to_file, filenames)
        else:
            filenames.append(file)

In [6]:
filenames_A001 = get_filename("CN/A001")
num_of_files_A001 = len(filenames_A001)
num_of_files_A001

8129

In [7]:
filenames_A001[:10]

['6959764600817_依能苏打水饮料_A_A001_15445.jpg',
 '6959764600817_依能苏打水饮料_A_A001_154845.jpg',
 '6959764600817_依能苏打水饮料_A_A001_15945.jpg',
 '6959764600817_依能苏打水饮料_A_A001_153045.jpg',
 '6959764600817_依能苏打水饮料_A_A001_154745.jpg',
 '6959764600817_依能苏打水饮料_A_A001_153345.jpg',
 '6959764600817_依能苏打水饮料_A_A001_151845.jpg',
 '6959764600817_依能苏打水饮料_A_A001_152345.jpg',
 '6959764600817_依能苏打水饮料_A_A001_153745.jpg',
 '6959764600817_依能苏打水饮料_A_A001_151145.jpg']

In [8]:
filenames_CN = get_filename("CN")
num_of_files_CN = len(filenames_CN)
num_of_files_CN

596457

## Get unique IDs (i.e. different products)

Assume the filename starts with a unique ID (which could be SKU, EAN, etc) followed by underscore. Given the path of a folder which may contain subfolders, get unique IDs (i.e., different products) in this folder.

In [9]:
def get_unique_ids(path):
    uids = set()
    get_unique_ids_help(path, uids)
    return uids

def get_unique_ids_help(path, uids):
    files = os.listdir(path)
    for file in files:
        path_to_file = os.path.join(path, file)
        if os.path.isdir(path_to_file):
            get_unique_ids_help(path_to_file, uids)
        else:
            uids.add(file.split("_")[0])

In [10]:
uids_CN = get_unique_ids("CN")
number_of_products_CN = len(uids_CN)
number_of_products_CN

22475

In [11]:
uids_A001 = get_unique_ids("CN/A001")
number_of_products_A001 = len(uids_A001)
number_of_products_A001

127

## Get the mapping from unique ID to product description

Assume the filename starts with a unique ID (which could be SKU, EAN, etc) followed by underscore and product description. Given the path of a folder which may contain subfolders, get the mapping from unique IDs to product description. The reason is that the unique ID here may be arbitrary, neither SKU nor EAN. 

In [12]:
def get_mapping_from_uid_to_description(path):
    uid_to_description = {}
    get_mapping_from_uid_to_description_help(path, uid_to_description)
    return uid_to_description

def get_mapping_from_uid_to_description_help(path, uid_to_description):
    files = os.listdir(path)
    for file in files:
        path_to_file = os.path.join(path, file)
        if os.path.isdir(path_to_file):
            get_mapping_from_uid_to_description_help(path_to_file, uid_to_description)
        else:
            s = file.split("_")
            if s[0] in uid_to_description:
                continue
            uid_to_description[s[0]] = s[1]

In [13]:
uid_to_description_CN = get_mapping_from_uid_to_description("CN")
assert len(uid_to_description_CN.keys()) == number_of_products_CN

In [14]:
uid_to_description_A001 = get_mapping_from_uid_to_description("CN/A001")
assert len(uid_to_description_A001.keys()) == number_of_products_A001

In [15]:
for k, v in list(uid_to_description_A001.items())[:5]:
    print(k + ": " + v)

6959764600817: 依能苏打水饮料
6920459939611: 康师傅喝开水550ml
6970920140022: 芙丝饮用天然矿泉水500ml
6901939641590: undefine
6901347800862: 椰树火山岩矿泉水5l


## Get mapping from unique ID to file location

Assume the filename starts with a unique ID (which could be SKU, EAN, etc) followed by underscore. Given the path of a folder which may contain subfolders, get the mapping from unique ID to a list of locations of the files containing the same product, so that we can split the files between training and testing for each product.

In [16]:
def get_mapping_from_uid_to_file_location(path):
    uid_to_file_locations = collections.defaultdict(list)
    get_mapping_from_uid_to_file_location_help(path, uid_to_file_locations)
    return uid_to_file_locations

def get_mapping_from_uid_to_file_location_help(path, uid_to_file_locations):
    files = os.listdir(path)
    for file in files:
        path_to_file = os.path.join(path, file)
        if os.path.isdir(path_to_file):
            get_mapping_from_uid_to_file_location_help(path_to_file, uid_to_file_locations)
        else:
            s = file.split("_")
            uid_to_file_locations[s[0]].append(path_to_file)

In [20]:
uid_to_file_locations_A001 = get_mapping_from_uid_to_file_location("CN/A001")
assert len(uid_to_file_locations_A001.keys()) == number_of_products_A001
assert sum([len(x) for x in uid_to_file_locations_A001.values()]) == num_of_files_A001

In [19]:
for k, v in list(uid_to_file_locations_A001.items())[:2]:
    print(k + ": ")
    for x in v:
        print(x)

6959764600817: 
CN/A001/6959764600817_依能苏打水饮料/6959764600817_依能苏打水饮料_A_A001_15445.jpg
CN/A001/6959764600817_依能苏打水饮料/6959764600817_依能苏打水饮料_A_A001_154845.jpg
CN/A001/6959764600817_依能苏打水饮料/6959764600817_依能苏打水饮料_A_A001_15945.jpg
CN/A001/6959764600817_依能苏打水饮料/6959764600817_依能苏打水饮料_A_A001_153045.jpg
CN/A001/6959764600817_依能苏打水饮料/6959764600817_依能苏打水饮料_A_A001_154745.jpg
CN/A001/6959764600817_依能苏打水饮料/6959764600817_依能苏打水饮料_A_A001_153345.jpg
CN/A001/6959764600817_依能苏打水饮料/6959764600817_依能苏打水饮料_A_A001_151845.jpg
CN/A001/6959764600817_依能苏打水饮料/6959764600817_依能苏打水饮料_A_A001_152345.jpg
CN/A001/6959764600817_依能苏打水饮料/6959764600817_依能苏打水饮料_A_A001_153745.jpg
CN/A001/6959764600817_依能苏打水饮料/6959764600817_依能苏打水饮料_A_A001_151145.jpg
CN/A001/6959764600817_依能苏打水饮料/6959764600817_依能苏打水饮料_A_A001_151245.jpg
CN/A001/6959764600817_依能苏打水饮料/6959764600817_依能苏打水饮料_A_A001_155045.jpg
CN/A001/6959764600817_依能苏打水饮料/6959764600817_依能苏打水饮料_A_A001_151345.jpg
CN/A001/6959764600817_依能苏打水饮料/6959764600817_依能苏打水饮料_A_A001_155445.jpg
CN/A00

## Issue of the data

The following two items have the same EAN but are in two subfolders and have different product description.

- CN/A001/6959764600817_依能苏打水饮料/6959764600817_依能苏打水饮料_A_A001_15545.jpg
- CN/A001/6959764600817_依能加锌苏打水500毫升/6959764600817_依能加锌苏打水500毫升_A_A001_107505006.jpg

之前提到的油漆，不同颜色的油漆是否具有相同的EAN？

## Train/test split

Given the path of a folder which may contain subfolders, split files of each product into two parts, one for training and the other for testing, with a specified ratio.  

In [148]:
def train_test_split(path, train_ratio=0.7, seed=43):
    """
    params:
    ratio: the proportion of training data
    """
    np.random.seed(seed)
    uid_to_file_location = get_mapping_from_uid_to_file_location(path)
    uid_to_description = get_mapping_from_uid_to_description(path)
    train_uid, train_description, train_file_location = [], [], []
    test_uid, test_description, test_file_location = [], [], []
    for uid in uid_to_file_location.keys():
        n = len(uid_to_file_location[uid])
        k = int(n * train_ratio)
        index = np.random.choice(n, k, replace=False)
        train_uid.extend([uid] * k)
        train_description.extend([uid_to_description[uid]] * k)
        train_file_location.extend([uid_to_file_location[uid][_] for _ in index])
        test_uid.extend([uid] * (n - k))
        test_description.extend([uid_to_description[uid]] * (n - k))
        test_file_location.extend([uid_to_file_location[uid][_] for _ in set(range(n)) - set(index)])
    return train_uid, train_description, train_file_location, test_uid, test_description, test_file_location

In [155]:
def test_train_test_split(path, train_uid, train_description, train_file_location, test_uid, test_description, test_file_location):
    num_of_files = get_number_of_files(path)
    assert len(train_uid) + len(test_uid) == num_of_files
    assert len(train_description) + len(test_description) == num_of_files
    assert len(train_file_location) + len(test_file_location) == num_of_files
    assert len(train_uid) == len(train_description) == len(train_file_location)
    assert len(test_uid) == len(test_description) == len(test_file_location)

In [159]:
train_uid, train_description, train_file_location, test_uid, test_description, test_file_location = train_test_split("CN/A001")

In [160]:
test_train_test_split("CN/A001", train_uid, train_description, train_file_location, test_uid, test_description, test_file_location)

In [151]:
len(train_uid)

5627

In [152]:
len(test_uid)

2502

## Save to json

In [161]:
def create_json_file(dst, uid, description, file_location):
    data = {}
    data["uid"] = uid
    data["description"] = description
    data["file_location"] = file_location
    with open(dst, 'w', encoding='utf-8') as out_file:
        json.dump(data, out_file, ensure_ascii=False)

In [162]:
create_json_file("A001_train.json", train_uid, train_description, train_file_location)

In [163]:
create_json_file("A001_test.json", test_uid, test_description, test_file_location)

## Clean data


## 微调数据集

- https://zhuanlan.zhihu.com/p/701930953
- https://modelscope.cn/datasets/modelscope/coco_2014_caption/summary

In [30]:
from modelscope.msdatasets import MsDataset
ds = MsDataset.load("coco_2014_caption", namespace="modelscope", split="train")
print(ds[0])

2024-06-14 09:12:50,831 - modelscope - INFO - PyTorch version 2.2.2 Found.
2024-06-14 09:12:50,834 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2024-06-14 09:12:50,981 - modelscope - INFO - Loading done! Current index file version is 1.15.0, with md5 137712c8ca5ebf0802c740a429d6cceb and a total number of 980 components indexed
/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


transformer is not installed, please install it if you want to use related modules


/opt/conda/lib/python3.10/site-packages/datasets/load.py:2524: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=no_checks' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/datasets/load.py:926: FutureWarning: The repository for coco_2014_caption contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at /root/.cache/modelscope/hub/datasets/modelscope/coco_2014_caption/master/meta/coco_2014_caption.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Generating train split: 414113 examples [00:13, 31531.41 examples/s]
Generating validation split: 40504 examples [00:01, 30367.29 examples/s]


{'uniq_id': '258768', 'image_id': '11195', 'caption': 'A snow skier assessing the mountain before starting to sky', 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480 at 0x791E21F79CF0>}


In [31]:
ds[:10]

{'uniq_id': ['258768',
  '725439',
  '580123',
  '283575',
  '392682',
  '154143',
  '142231',
  '569327',
  '434237',
  '480315'],
 'image_id': ['11195',
  '367065',
  '213997',
  '370736',
  '524048',
  '427137',
  '579907',
  '45468',
  '35230',
  '284155'],
 'caption': ['A snow skier assessing the mountain before starting to sky',
  'a guy that is brushing his teeth and a baby too',
  'A chair and a fireplace in a room.',
  'A red street sign showing Volt street and Mill street.',
  'A young man standing next to a pile of pallets.',
  'Some sheep walking in the field and having some fun.',
  'A living room complete with couches, television, and fireplace.',
  'a bath room with a toilet and a sink and a pile of trash',
  "Two pizza's salad's and drinks on a table with plates.",
  'A clean kitchen with white cabinets and a black oven'],
 'image': [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480>,
  <PIL